In [9]:
# read json summaries file
# download cif files
# save cif files in a folder

import json
import os
import urllib.request
from tqdm import tqdm

# read json file
with open('summaries.json') as f:
    summaries = json.load(f)
    temp = {val : key for key, val in summaries.items()}
    summaries = {val : key for key, val in temp.items()}

# create folder
if not os.path.exists('cifs'):
    os.makedirs('cifs')

import pandas as pd
data = pd.read_csv("cod_new.csv")

/var/folders/gl/2h8nn4ps031g1jysgll17wz00000gn/T/ipykernel_42835/1691248450.py:21: DtypeWarning: Columns (23,24,25,30,31,32,38,44,45,46,48,49,51,52,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cod_new.csv")


In [10]:
data = data[data.doi.isin(list(dict.keys(summaries)))]
data = data[data.doi.notnull()]
data = data[data.doi != '']

In [11]:
# add description summaries value to dataframe
data['summary'] = data['doi'].map(summaries)
data = data[data.summary.notnull()]
data = data[data.summary != '']
data = data[['file', 'doi', 'summary']]

In [12]:
import re
def clean(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

data['summary'] = data['summary'].apply(clean)

In [13]:
data = data.reset_index()

In [17]:
data.to_csv('cod_processed.csv', index=False)

In [47]:


# download cif files
for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    #urllib.request.urlretrieve(f"http://www.crystallography.net/cod/{row['file']}.cif", f"cifs/{row['file']}.cif")
    # robust error handling
    try:
        urllib.request.urlretrieve(f"http://www.crystallography.net/cod/{row['file']}.cif", f"cifs/{row['file']}.cif")
    except:
        pass

  2%|███▍                                                                                                                                                                                   | 777/41439 [21:02<18:21:06,  1.62s/it]


KeyboardInterrupt: 

In [53]:
# save dataframe without title
data[["file", "summary"]].to_csv("cifs/id_prop.csv", index=False, header=False)

# make a copy of atom_init.json in cif dir
import shutil
shutil.copyfile('atom_init.json', 'cifs/atom_init.json')

'cifs/atom_init.json'